In [5]:
import pandas as pd
import ast
from collections import Counter
import random
from itertools import compress
from tqdm import tqdm
from scipy import stats

In [19]:
power = pd.read_csv("/content/output_1025.csv")
agency_power = pd.read_csv("/content/agency_power_lemma.csv")

In [150]:
def calculate_power(agent,patient):
  # calculate the power of a character using the power frames lexicon
  # input: agent_dict and patient_dict
  length = 0
  power = 0

  # verbs for which the character is the agent
  for i in list(agent.keys()):
    length += agent[i]
    if i in list(agency_power["verb"]):
      index = list(agency_power["verb"]).index(i)
      if index != -1:
        if agency_power["power"][index] == "power_agent":
          power += 1 * agent[i]
        elif agency_power["power"][index] == "power_theme":
          power -= 1 * agent[i]

  # verbs for which the character is the patient
  for i in list(patient.keys()):
    length += patient[i]
    if i in list(agency_power["verb"]):
      index = list(agency_power["verb"]).index(i)
      if index != -1:
        if agency_power["power"][index] == "power_agent":
          power -= 1 * patient[i]
        elif agency_power["power"][index] == "power_theme":
          power += 1 * patient[i]
  try:
    normalized_power = power/length # normalize the power
  except:
    normalized_power = "/"

  return normalized_power

In [151]:
my_power = power

In [105]:
power = my_power[my_power.stage == 1].reset_index(drop = True) # change the stage number here to choose the stage

In [152]:
counter_agent = Counter()
counter_patient = Counter()

In [ ]:
power

In [ ]:
power = power[[i.startswith("{") for i in power.agent]].reset_index(drop=True)
power = power[[i.startswith("{") for i in power.patient]].reset_index(drop=True)
power

In [98]:
power = power[['name', 'gender','genre','power','agent','patient','stage']]

In [155]:
# the whole collections of verbs divided by agent and patient
for i in power.agent:
  counter_agent.update(ast.literal_eval(i))

for i in power.patient:
  counter_patient.update(ast.literal_eval(i))

In [156]:
agent_list = []
for key,value in counter_agent.items():
  agent_list.extend([key]*value)

patient_list = []
for key,value in counter_patient.items():
  patient_list.extend([key]*value)


In [157]:
len(agent_list),len(patient_list)

(60052, 10957)

In [158]:
# number of verbs for each character
num_verb = [sum(ast.literal_eval(power.agent[i]).values())+sum(ast.literal_eval(power.patient[i]).values()) for i in range(len(power))]

In [159]:
# indices to randomly choose verbs for each character
idx = []
for i in range(len(num_verb)):
  idx.extend([i]*num_verb[i])

In [160]:
# randomly shuffle the indices
random.shuffle(idx)
agent_idx = idx[:len(agent_list)]
patient_idx = idx[len(agent_list):]

In [161]:
def validate(num):
  power[f"validation{num}"] = 0;
  random.shuffle(idx)
  agent_idx = idx[:len(agent_list)]
  patient_idx = idx[len(agent_list):]
  for i in tqdm(range(len(power))):
    select_patient = [x == i for x in patient_idx]
    select_agent = [x == i for x in agent_idx]

    the_patient = list(compress(patient_list, select_patient))
    the_agent = list(compress(agent_list, select_agent))

    agent_dict = dict(Counter(the_agent))
    patient_dict = dict(Counter(the_patient))

    power.loc[i,f"validation{num}"] = calculate_power(agent_dict,patient_dict)


In [ ]:
for i in range(100):
  # run 100 validations!
  validate(i)

In [165]:
power.to_csv("/content/power_stage1_validation.csv",index=False)

## start validation



In [ ]:
power1 = pd.read_csv("/content/power_stage1_validation.csv")
power2 = pd.read_csv("/content/power_stage2_validation.csv")
power3 = pd.read_csv("/content/power_stage3_validation.csv")
power4 = pd.read_csv("/content/power_stage4_validation.csv")
power5 = pd.read_csv("/content/power_stage5_validation.csv")

In [231]:
power = power1 # change the number to get validate the stage of interest

In [232]:
p_f = power[power.gender == "she/her"]
p_m = power[power.gender == "he/him/his"]

In [233]:
val_names = []
for i in p_f.columns:
  if i.startswith("validation"):
    val_names.append(i)
len(val_names)

100

In [234]:
whole_f = [p_f[i].mean() for i in val_names]
curr_mean_f = p_f.power.mean()
whole_m = [p_m[i].mean() for i in val_names]
curr_mean_m = p_m.power.mean()

In [235]:
import numpy as np
np.mean(whole_f),np.mean(whole_m) # mean of simulated means

(0.20337900283921723, 0.2014233454944071)

In [236]:
from statistics import stdev
from scipy.stats import t

0.13292113053102939

In [237]:
# the p values
print(t.cdf((curr_mean_f-np.mean(whole_f))/stdev(whole_f),100)*2)
print((t.cdf((curr_mean_m-np.mean(whole_m))/stdev(whole_m),100))*2)
# print((1-t.cdf((curr_mean_m-np.mean(whole_m))/stdev(whole_m),100))*2) # use this for stage 5 male

0.9721417088508926
0.8083846499835313


In [ ]:
curr_mean_m, curr_mean_f # calculated means